# 把title包含关键词的条目放入categories文件夹中，并生成对应的文件夹，并把已经下载好的文件放入其中

In [16]:
import pandas as pd
csv_file = 'data.csv'
data = pd.read_csv(csv_file)
data = data.dropna()
# 根据title筛选
keyword = 'car'  
# 可以包含多个关键词，用'-'分隔，且不要包含空格，例如   desk-table
# 如果是多个关键词的话，生成的文件夹就是desk-table，而不是两者各一个，建议对于一个类别用多个同义词作为关键词

In [17]:
pattern = []
for word in keyword.split('-'):
    if word != '':
        word = word.strip()
        pattern.append(r'\b{}\b'.format(word))
pattern = '|'.join(pattern)
print(pattern)
minidata = data[data['title'].str.contains(pattern, regex=True)]
print('包含目标信息的条目数量：', minidata.shape[0])
print('已经下载的：', minidata[minidata['is_download'] == 'yes'].shape[0])
minidata

\bcar\b
包含目标信息的条目数量： 67
已经下载的： 67


,Unnamed: 0.1,Unnamed: 0,page,id,title,cc,download_times,like,is_download,size(MB),path
512,512,512,29,2194,Lego Police car 8152,CC-BY-SA,390,0,yes,0.2384929656982422,data/2194.blend
2741,2741,2741,153,4502,Orca concept car,CC-BY,644,9,yes,4.2466583251953125,data/4502.blend
2978,2978,2978,166,4745,Stone-age two seater car,CC-0,482,7,yes,0.6125659942626953,data/4745.blend
3132,3132,3132,175,4904,Aluminium car wheel with tire,CC-BY,1179,0,yes,0.1685810089111328,data/4904.blend
4234,4234,4234,236,6018,car and road,CC-BY,957,4,yes,2.3336610794067383,data/6018.blend
...,...,...,...,...,...,...,...,...,...,...,...
22449,22449,22450,1248,26373,Basic Jeep car model,CC-0,688,19,yes,5.811988830566406,data/26373.blend
23604,23604,23605,1312,28462,low poly car,CC-0,54,0,yes,2.618907928466797,data/28462.blend
24095,24095,24096,1339,29311,Blue low poly car,CC-BY,139,0,yes,0.16785526275634766,data/29311.blend
24133,24133,24134,1341,29385,Llanta wheel Tire car,CC-BY,277,7,yes,1.688201904296875,data/29385.blend


In [18]:
# 将这些条目整理，并放入categories中
print('keyword:',keyword)
# 文件复制
import os
import shutil
# 创建文件夹，用对应的keyword命名
folder_path = 'categories/{}'.format(keyword)
os.makedirs(folder_path, exist_ok=True)
folder_data_path = 'categories/{}/data'.format(keyword)
os.makedirs(folder_data_path, exist_ok=True)
is_download_df = minidata[minidata['is_download'] == 'yes']
# 逐条复制文件
for i in range(is_download_df.shape[0]):
    line = is_download_df.iloc[i]
    path = line['path']
    target_path = os.path.join(folder_path, path)
    shutil.copy(path, target_path)
# 创建该条目的csv文件
minidata.to_csv(os.path.join(folder_path, 'data.csv'), index=False)



keyword: car


# 错误处理
某些时候因为下载次数过多ip别封禁，这时候不会提示下载cookie错误，而是无论对于什么模型，网站返回的下载都是一个192字节的文件，具体大小为0.00018310546875MB，我们要将这些条目删掉

In [136]:
is_download_df = data[data['is_download'] == 'yes']
print("处理之前的尺寸：", is_download_df.shape[0])
is_download_df['size(MB)'] = is_download_df['size(MB)'].astype(float)
condition = is_download_df['size(MB)'] == 0.00018310546875
is_download_df.loc[condition, ['is_download', 'size(MB)', 'path']] = 'no'
print("处理之前的尺寸：", is_download_df[is_download_df['is_download'] == 'yes'].shape[0])
# 覆盖原表中的这些内容
data.loc[is_download_df.index] = is_download_df
data.to_csv('data.csv')

处理之前的尺寸： 14118
处理之前的尺寸： 14118


/tmp/ipykernel_3635987/3343344453.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  is_download_df['size(MB)'] = is_download_df['size(MB)'].astype(float)


# test
不要运行这里的内容

In [147]:
import re
def contains1(text, keyword):
    pattern = r'\b{}\b'.format(keyword)  # 这样的匹配策略只会匹配单独的单词，如果是一个单词中包含了关键词匹配失败
    match = re.search(pattern, text)
    if match:
        return True
    else:
        return False

def contains(text, keyword:str):
    if keyword=='':
        return True
    words = keyword.split(',')
    res = False
    for word in words:
        res = res or contains1(text, word)
    return res

contains('low poly ca des', 'car,desk')

False

In [6]:
# 为data.csv添加一列，nerf_data_path
import pandas as pd
csv_file = 'data.csv'
data = pd.read_csv(csv_file)

In [7]:
data['nerf_data_path'] = 'no'

In [10]:
data.to_csv(csv_file,index=None)